In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document

docs = [
    Document(page_content="Langchin helps build LLM applications"),
    Document(page_content="Pinecone is a vector database for semantic search"),
    Document(page_content="The Eiffel Tower is located in Paris."),
    Document(page_content="Langchain can be used to develop agentic ai application."),
    Document(page_content="Langchain has many types of retrivers.")
]

embedding_model = OpenAIEmbeddings()
dense_vectorstore = FAISS.from_documents(docs, embedding_model)
dense_retriever = dense_vectorstore.as_retriever()

# sparse Retriever (BM25)
sparse_retriever = BM25Retriever.from_documents(docs)
sparse_retriever.k = 3

hybrid_retriever = EnsembleRetriever(retrievers=[dense_retriever, sparse_retriever], 
                                     weights=[0.7, 0.3]
                                     )



In [3]:
hybrid_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000146D1FC7790>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x00000146D2697A50>, k=3)], weights=[0.7, 0.3])

In [7]:
query = "How can I build applications with LLMs?"
result = hybrid_retriever.invoke(query)

for i, doc in enumerate(result):
    print(f"\nDocument{i+1}:\n{doc.page_content}")


Document1:
Langchin helps build LLM applications

Document2:
Langchain can be used to develop agentic ai application.

Document3:
Langchain has many types of retrivers.

Document4:
Pinecone is a vector database for semantic search


### RAG Pipeline with Hybrid Retriever

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

prompt = PromptTemplate.from_template("""
Answer the question based on the context below.
Context:
{context}

Question:
{input}

Answer:
""")

llm = ChatOpenAI(model='gpt-4o', temperature=0.2)

# Stuff documents chain
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

# Full RAG chain
rag_chain = create_retrieval_chain(retriever=hybrid_retriever, combine_docs_chain=document_chain)

query = {"input": "How can I build an app using LLMs"}
response = rag_chain.invoke(query)

print(f"Q. {query['input']}")
print(f"A. {response['answer']}")


Q. How can I build an app using LLMs
A. To build an app using Large Language Models (LLMs), you can follow these general steps:

1. **Define the Purpose**: Clearly outline what you want your app to achieve. Determine the specific tasks or problems the LLM will address.

2. **Choose the Right Tools**: Utilize frameworks like Langchain, which is designed to help build applications with LLMs. Langchain can assist in developing agentic AI applications and offers various retrievers to enhance functionality.

3. **Select a Vector Database**: For efficient semantic search and retrieval, consider using a vector database like Pinecone. This will help in managing and querying large datasets effectively.

4. **Data Preparation**: Gather and preprocess the data your application will use. This may involve cleaning, formatting, and structuring the data to ensure it is suitable for the LLM.

5. **Model Selection**: Choose an appropriate LLM that fits your application's needs. This could be an existin